In [1]:
from dotenv import load_dotenv
import os

from datetime import datetime

# Load environment variables from .env file
load_dotenv()


True

In [2]:
from langchain_groq import ChatGroq
GROQ_LLM=ChatGroq(model='llama3-70b-8192')

In [11]:
from crewai import Task, Process
from crewai_tools import tool



In [12]:
# from crewai_tools import BaseTool



# class MyCustomTool(BaseTool):

#     name: str = "Name of my tool"

#     description: str = "Clear description for what this tool is useful for, you agent will need this information to use it."



#     def _run(self, argument: str) -> str:

#         # Implementation goes here

#         return "Result from custom tool"

# from crewai_tools import tool

# @tool("Name of my tool")

# def my_tool(question: str) -> str:

#     """Clear description for what this tool is useful for, you agent will need this information to use it."""

#     # Function logic here

#     return "Result from your custom tool"
@tool("Tool Name")
def generate_kpath(poscar_str: str) -> str:
    """This tools is used to generate the kpath for band structure calculation in VASP. This will return the a string with the contents of the KPOINTS file."""

    from pymatgen.io.vasp.inputs import Poscar
    from pymatgen.symmetry.kpath import KPathSeek
    
    poscar=Poscar.from_str(poscar_str)
    structure = poscar.structure
    kpath_seek=KPathSeek(structure)

    kpath=kpath_seek.kpath
    kpoints=kpath['kpoints']
    paths=kpath['path']

    kpoints_str="generate KPOINTS by seekpath\n"
    kpoints_str+="50 ! Grid points\n"
    kpoints_str+="Line_points\n"
    kpoints_str+="reciprocal\n"
    for path in paths:
        for i,kpoint_name in enumerate(path):
            if i == len(path)-1:
                break
            current_kpoint_name=path[i]
            current_kpoint=str(kpoints[current_kpoint_name]).replace('[','').replace(']','').replace(',','')
            
            next_kpoint_name=path[i+1]
            next_kpoint=str(kpoints[next_kpoint_name]).replace('[','').replace(']','').replace(',','')

            kpoints_str+= current_kpoint + f" ! {current_kpoint_name}\n"
            kpoints_str+= next_kpoint + f" ! {next_kpoint_name}\n"
            kpoints_str+="\n"


    return kpoints_str


In [26]:
from crewai import Crew, Agent

class VaspAgents:

    # def create_chat_agent(self):
    #     return Agent(
    #         role='VASP Input Writer', 
    #         goal='Your goal is to create the necessary VASP input files for the calculation',
    #         backstory='You are a VASP expert and you have been tasked with creating the necessary VASP input files for the calculation',
    #         # tools=[my_tool1, my_tool2],
    #         llm=GROQ_LLM,
    #         verbose=True,
    #         allow_delegation=False,
    #         max_iter=5,
    #         memory=True,
    #         step_callback=lambda x: print(x, "VASP Input Creator")
    #         )

    def create_kpoints_writer(self):
        return Agent(
            role='VASP KPOINTS Writer', 
            goal='Your goal is to create an the appropiate KPOINTS file for a VASP calculation. You can either generate kpoints path or a mesh',
            backstory='You are a VASP expert and you have been tasked with writing an KPOINTS file for the calculation',
            tools=[generate_kpath],
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            step_callback=lambda x: print(x, "VASP KPOINTS Writer")
            )

    def create_incar_writer(self):
        return Agent(
            role='VASP INCAR Writer', 
            goal='Your goal is to create an the correct INCAR file for a VASP calculation',
            backstory='You are a VASP expert and you have been tasked with writing an INCAR file for the calculation',
            llm=GROQ_LLM,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            step_callback=lambda x: print(x, "VASP INCAR Writer")
            )




In [48]:
class VaspTasks:
    def __init__(self,poscar_str):
        self.poscar_str=poscar_str

    def draft_incar(self, agent):
        return Task(
            description="""Write an INCAR file for a VASP calculation corresponding to the POSCAR file: {poscar_str}.\n It should leave only appropriate tags in the INCAR file""",
            expected_output=f"""An INCAR file. It should leave only the appropriate tags for the calculation.""",
            context=[],
            agent=agent,
            output_file='INCAR',
        )
    def draft_kpoints_path(self,agent):
        return Task(
            description="""Write a KPOINTS file for a VASP calculation corresponding to the POSCAR file: {poscar_str}""",
            expected_output=f"""A KPOINTS file""",
            context=[],
            agent=agent,
            output_file='KPOINTS',
        )
    
    # def draft_poscar(self):
    #     return Task(
    #         description=f"""Write a POSCAR file for a VASP calculation""",
    #         expected_output=f"""A POSCAR file""",
    #         context=[],
    #         agent=VaspAgents().create_poscar_agent(),
    #         output_file='POSCAR',
    #     )
    
    # def draft_potcar(self):
    #     return Task(
    #         description=f"""Write a POTCAR file for a VASP calculation""",
    #         expected_output=f"""A POTCAR file""",
    #         context=[],
    #         agent=VaspAgents().create_potcar_agent(),
    #         output_file='POTCAR',
    #     )
  

In [49]:
poscar=""" Sr V O
1.0
   3.8465199999999999   0.0000000000000000   0.0000000000000000
   0.0000000000000000   3.8465199999999999   0.0000000000000000
   0.0000000000000000   0.0000000000000000   3.8465199999999999
 Sr V O
 1 1 3
Direct
   0.0000000000000000   0.0000000000000000   0.0000000000000000
   0.5000000000000000   0.5000000000000000   0.5000000000000000
   0.5000000000000000   0.5000000000000000   0.0000000000000000
   0.5000000000000000   0.0000000000000000   0.5000000000000000
   0.0000000000000000   0.5000000000000000   0.5000000000000000
"""

In [50]:
agents=VaspAgents()
tasks=VaspTasks(poscar)

## Agents
# chat_agent=agents.create_chat_agent()
kpoints_agent=agents.create_kpoints_writer()
incar_agent=agents.create_incar_writer()

# Tasks
draft_incar=tasks.draft_incar(agent=incar_agent)
draft_kpoints=tasks.draft_kpoints(agent=kpoints_agent)
# draft_poscar=tasks.draft_poscar()

In [51]:
crew = Crew(
    agents=[kpoints_agent, incar_agent],
    tasks=[draft_kpoints, draft_incar],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=True,
    step_callback=lambda x: print(x, "MasterCrew Agent")
)

2024-05-08 12:05:42,837 - 4680 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [52]:
results=crew.kickoff({'poscar_str':poscar})

#Print the results
print("Crew Work Results")
print(results)

#

print(crew.usage_metrics)

 [DEBUG]: == Working Agent: VASP KPOINTS Writer
 [INFO]: == Starting Task: Write a KPOINTS file for a VASP calculation corresponding to the POSCAR file:  Sr V O
1.0
   3.8465199999999999   0.0000000000000000   0.0000000000000000
   0.0000000000000000   3.8465199999999999   0.0000000000000000
   0.0000000000000000   0.0000000000000000   3.8465199999999999
 Sr V O
 1 1 3
Direct
   0.0000000000000000   0.0000000000000000   0.0000000000000000
   0.5000000000000000   0.5000000000000000   0.5000000000000000
   0.5000000000000000   0.5000000000000000   0.0000000000000000
   0.5000000000000000   0.0000000000000000   0.5000000000000000
   0.0000000000000000   0.5000000000000000   0.5000000000000000



> Entering new CrewAgentExecutor chain...
Thought: I need to generate a KPOINTS file for a VASP calculation using the provided POSCAR file.

Action: Tool Name
Action Input: {"poscar_str": "Sr V O\n1.0\n   3.8465199999999999   0.0000000000000000   0.0000000000000000\n   0.0000000000000000   3.84651